# Multi-Solution Generation: Using SymK in the Unified Planning Library

In [1]:
import sys
from collections import defaultdict

import unified_planning as up
import up_symk
from unified_planning.shortcuts import *
from unified_planning.io import PDDLReader

## A simple example

We use the iconic gripper domain, where the goal is to move balls from room A to room B with a gripper that has two arms, which we read in as pddl files.

In [7]:
reader = PDDLReader()
pddl_problem = reader.parse_problem("gripper-domain.pddl", "gripper-prob01.pddl")
pddl_problem.add_quality_metric(MinimizeSequentialPlanLength())

## Validatior
We define two functions that serve as helper functions to evaluate the found plans and calculate the cost of the plans.

In [ ]:
def get_plan_cost(problem, plan):
    pv = PlanValidator(problem_kind=problem.kind)
    pv_res = pv.validate(problem, plan)
    return pv_res.metric_evaluations[problem.quality_metrics[0]]

def get_plans_by_cost(problem, plans):
    pv = PlanValidator(problem_kind=problem.kind)
    plans_by_cost = defaultdict(lambda: [])
    for plan in plans:
        pv_res = pv.validate(problem, plan)
        cost = pv_res.metric_evaluations[problem.quality_metrics[0]]
        plans_by_cost[cost].append(result.plan)
    return plans_by_cost

## Finding a single optimal solution
We solve our gripper problem optimally with SymK.

In [9]:
with OneshotPlanner(name='symk') as planner:
    result = planner.solve(pddl_problem) # output_stream=sys.stdout
    if result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
        cost = get_plan_cost(pddl_problem, result.plan)
        print(f"{planner.name} found this plan: {result.plan} with cost {cost}.")
    else:
        print("No plan found.")

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_79998/3546280737.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent optimal and top-k planner.

SymK found this plan: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball2, roomb, right)
    drop(ball1, roomb, left)
    move(roomb, rooma)
    pick(ball3, rooma, left)
    pick(ball4, rooma, right)
    move(rooma, roomb)
    drop(ball3, roomb, left)
    drop(ball4, roomb, right) with cost 11.


## Finding multiple optimal solution
We solve our gripper problem with SymK and ask for three optimal solutions, which are reported.

In [10]:
plans = []

with AnytimePlanner(name='symk-opt', params={"number_of_plans": 3}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            plans.append(result.plan)
            print(f"Plan {i+1}: {result.plan} with cost {get_plan_cost(pddl_problem, result.plan)}.")
            print()
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            for cost, plans in get_plans_by_cost(pddl_problem, plans).items():
                print(f"{planner.name} found {len(plans)} optimal plans with cost {cost}.")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.") 

  *** Credits ***
  * In operation mode `AnytimePlanner` at line 3 of `/tmp/ipykernel_79998/1171804862.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent optimal and top-k planner.

Plan 1: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball2, roomb, right)
    drop(ball1, roomb, left)
    move(roomb, rooma)
    pick(ball3, rooma, left)
    pick(ball4, rooma, right)
    move(rooma, roomb)
    drop(ball3, roomb, left)
    drop(ball4, roomb, right) with cost 11.

Plan 2: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball2, roomb, right)
    drop(ball1, roomb, left)
    move(roomb, rooma)
    pick(ball3, rooma, left)
    pick(ball4, rooma, right)
    move(rooma, roomb)
    drop(ball4, roomb, righ

## Finding all optimal solution
We find all optimal solutions using SymK. We report the number of existing optimal plans and their cost.

In [11]:
# Query an anytime planner with OPTIMAL_PLANS guarantee and generate all optimal plans => SymK

plans = []

with AnytimePlanner(problem_kind=pddl_problem.kind, anytime_guarantee="OPTIMAL_PLANS") as planner:
    print(planner.name)
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            plans.append(result.plan)
            if i > 0 and i % 100 == 0:
                print(f"{planner.name} found {i} plans...")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print()
            print(f"{planner.name} found {i} plans!")
            print()
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.")
            
# Calculate the cost of the plans found
print("Calculate the cost of the plans found...")
for cost, plans in get_plans_by_cost(pddl_problem, plans).items():
    print(f"{planner.name} found {len(plans)} plans with cost {cost}.")

  *** Credits ***
  * In operation mode `AnytimePlanner` at line 5 of `/tmp/ipykernel_79998/198799663.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent optimal and top-k planner.

SymK (with optimality guarantee)
SymK (with optimality guarantee) found 100 plans...
SymK (with optimality guarantee) found 200 plans...
SymK (with optimality guarantee) found 300 plans...

SymK (with optimality guarantee) found 384 plans!

Calculate the cost of the plans found...
SymK (with optimality guarantee) found 384 plans with cost 11.


## Find multiple solutions ordered by cost
Again, we solve our gripper problem and ask for 500 plans, from which 384 plans are optimal with cost 11 and the remaining 116 of

In [6]:
# Find 500 plans with SymK
plans = []

with AnytimePlanner(name='symk', params={"number_of_plans": 500}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            plans.append(result.plan)
            if i > 0 and i % 100 == 0:
                print(f"{planner.name} found {i} plans...")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print()
            print(f"{planner.name} found {i} plans!")
            print()
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.")

# Calculate the cost of the plans found
print("Calculate the cost of the plans found...")
for cost, plans in get_plans_by_cost(pddl_problem, plans).items():
    print(f"{planner.name} found {len(plans)} plans with cost {cost}.")

  *** Credits ***
  * In operation mode `AnytimePlanner` at line 4 of `/tmp/ipykernel_79998/2464694441.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent optimal and top-k planner.

SymK found 100 plans...
SymK found 200 plans...
SymK found 300 plans...
SymK found 400 plans...

SymK found 500 plans!

Calculate the cost of the plans found...
SymK found 384 plans with cost 11.
SymK found 116 plans with cost 12.
